colab 환경에서 실행

# 전처리

## install packages

In [2]:
!pip install transformers
!pip install hanja
!pip install tensorflow_addons

     |████████████████████████████████| 2.6 MB 8.3 MB/s 
     |████████████████████████████████| 895 kB 59.1 MB/s 
     |████████████████████████████████| 3.3 MB 59.9 MB/s 
     |████████████████████████████████| 636 kB 58.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 120 kB 8.2 MB/s 
     |████████████████████████████████| 265 kB 13.7 MB/s 
     |████████████████████████████████| 280 kB 14.3 MB/s 
     |████████████████████████████████| 242 kB 15.4 MB/s 
     |████████████████████████████████| 242 kB 18.2 MB/s 
     |████████████████████████████████| 242 kB 27.4 MB/s 
     |████████████████████████████████| 229 kB 29.0 MB/s 
     |████████████████████████████████| 229 kB 38.2 MB/s 
INFO: pip is looking at multiple versions of pytest-cov to determine which version is compatible with other requirements. This could take a while.
  Created 

In [2]:
from tensorflow_addons.optimizers import AdamW

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
#from konlpy.tag import Mecab
import re
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from transformers import ElectraTokenizer
from transformers import TFElectraForSequenceClassification
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import hanja
from hanja import hangul

In [4]:
train = pd.read_csv('../data/train_data.csv',index_col=0).reset_index(drop=True)
test = pd.read_csv('../data/test_data.csv',index_col=0).reset_index(drop=True)

In [5]:
# 한자를 한글로 변환, 특수문자 제거
def prepare_text(data):
    transfer = []
    for index, row in data.iterrows():
        result = hanja.translate(row['title'], 'substitution') # transfer hanja to korean
        con = re.sub(r"[^a-zA-Z0-9가-힣]"," ",result) # remove Special Characters
        transfer.append(con)
    return transfer

## prepare data

In [6]:
train['title'] = prepare_text(train)
test['title'] = prepare_text(test)

In [8]:
train.tail(20)

,title,topic_idx
45634,NBA 미네소타 뎅 영입 팀버울브스 아니고 팀버불스,5
45635,한국무용협회 예술대상에 박명숙 김근희 김운미 박재근,3
45636,기고 아인슈타인 지휘에 맞춰 블랙홀 듀엣 연주가 시작됐다,3
45637,1보 코로나19 확진자 총 1만156명 어제 하루 94명 증가,2
45638,일자민당 원로 헌법9조는 세계유산 개정 바늘귀만큼도 안돼,4
45639,민주일반연맹 비정규직 차별철폐 공동행동,2
45640,전국대학원생노조 경북대 실험실 폭발 피해자 끝까지 책임져야,2
45641,창원 시내버스 노조 파업 투표 가결 30일 운행 멈춘다종합,2
45642,장석주 시인 사랑은 새로운 이야기가 잉태되는 순간,3
45643,남친과 함께 잔혹하게 동거인 학대 20대 여성 구속영장,2


In [9]:
test.head()

,title
0,유튜브 내달 2일까지 크리에이터 지원 공간 운영
1,어버이날 맑다가 흐려져 남부지방 옅은 황사
2,내년부터 국가RD 평가 때 논문건수는 반영 않는다
3,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것
4,회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간


## tokenize

In [10]:
MAX_LEN = 45
BATCH_SIZE = 32
NUM_EPOCHS = 5

In [11]:
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

In [12]:
def electra_tokenizer(sent, MAX_LEN):
    encoded_dict = tokenizer.encode_plus(
        text = sent,
        add_special_tokens = True,      # Add '[CLS]' and '[SEP]'
        max_length = MAX_LEN,           # Pad & truncate all sentences.
        pad_to_max_length = True,
        return_attention_mask = True,
    )
    
    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask'] # And its attention mask (simply differentiates padding from non-padding).
    token_type_id = encoded_dict['token_type_ids'] # differentiate two sentences
    
    return input_id, attention_mask, token_type_id

In [14]:
def tokenize_electra(data):
    # token id, attention mask, token type id를 최종 return
    input_ids = []
    attention_masks = []
    token_type_ids = []
    
    for data_sent in tqdm(data["title"]):
        try:
            input_id, attention_mask, token_type_id = electra_tokenizer(data_sent, MAX_LEN)
          
            input_ids.append(input_id)
            attention_masks.append(attention_mask)
            token_type_ids.append(token_type_id)

        except Exception as e:
            print(e)
            print(data_sent)
            pass
    return np.array(input_ids,dtype=int), np.array(attention_masks,dtype=int), np.array(token_type_ids,dtype=int)

In [15]:
train_input_ids, train_attention_masks, train_token_type_ids = tokenize_electra(train)
train_labels = np.asarray(train['topic_idx'],dtype=np.float32)

train_inputs = (train_input_ids, train_attention_masks, train_token_type_ids)

  0%|          | 0/45654 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 45654/45654 [00:12<00:00, 3758.45it/s]


In [16]:
test_input_ids, test_attention_masks, test_token_type_ids = tokenize_electra(test)

test_inputs = (test_input_ids, test_attention_masks, test_token_type_ids)

  0%|          | 0/9131 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 9131/9131 [00:02<00:00, 3491.60it/s]


# Model build : 감성분석 미세조정

In [23]:
model = TFElectraForSequenceClassification.from_pretrained('monologg/koelectra-base-v3-discriminator',from_pt=True, num_labels=7, attention_probs_dropout_prob=0.3, hidden_dropout_prob=0.3)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'electra.embeddings.position_ids']
- This IS expected if you are initializing TFElectraForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFElectraForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dens

In [28]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

optimizer = AdamW(learning_rate=1e-5, weight_decay=False)

earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=2)

In [27]:
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [29]:
model.fit(train_inputs, train_labels, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, validation_split = 0.3, callbacks=[earlystop_callback])

Epoch 1/5
999/999 [==============================] - 373s 358ms/step - loss: 1.6330 - accuracy: 0.3446 - val_loss: 1.2329 - val_accuracy: 0.4916
Epoch 2/5
999/999 [==============================] - 355s 355ms/step - loss: 0.7981 - accuracy: 0.7348 - val_loss: 0.6438 - val_accuracy: 0.7904
Epoch 3/5
999/999 [==============================] - 353s 354ms/step - loss: 0.5309 - accuracy: 0.8339 - val_loss: 0.5269 - val_accuracy: 0.8322
Epoch 4/5
999/999 [==============================] - 349s 349ms/step - loss: 0.4712 - accuracy: 0.8501 - val_loss: 0.4610 - val_accuracy: 0.8490
Epoch 5/5
999/999 [==============================] - 348s 349ms/step - loss: 0.4291 - accuracy: 0.8617 - val_loss: 0.4783 - val_accuracy: 0.8431


In [30]:
# 영어 포함, attention과 hidden layer에 dropout 적용
results = model.predict(test_inputs)
topic = []
for i in range(len(results)):
    topic.append(np.argmax(results[i],axis=1))

submission = pd.read_csv('../data/sample_submission.csv')
submission['topic_idx'] = np.array(topic).T
submission.to_csv('../submission/electra_8.csv', index=False)

In [ ]:
model.save('../electra5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /content/drive/MyDrive/dacon/뉴스 토픽 분류/electra5/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/dacon/뉴스 토픽 분류/electra5/assets


In [31]:
np.save('../submission/electra8_save', results['logits'])

In [ ]:
# elctra1 : epoch5, lr=1e-5, dropout 각각 0.3